In [1]:
# https://towardsdatascience.com/deeplearning-with-tabular-data-data-processing-cd2e73908257

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
from scipy import stats
from scipy.stats import zscore
from math import radians, cos, sin, asin, sqrt
import pydot_ng as pydot
import keras
from keras import metrics
from keras import regularizers
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.optimizers import Adam, RMSprop
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.utils import plot_model

import tensorflow as tf

In [3]:
depev_final = pd.read_csv('depev_final.csv')
depev_final.drop(depev_final.columns[depev_final.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
depev_final.head()

,URBRRL,RATCAT_A,INCGRP_A,INCTCFLG_A,FAMINCTC_A,IMPINCFLG_A,PPSU,PSTRAT,HISPALLP_A,RACEALLP_A,...,ANGEV_A,CHDEV_A,CHLEV_A,HYPEV_A,PHSTAT_A,AVAIL_A,INTV_QRT,WTFA_A,WTIA_A,POVRATTC_A
0,4,9,3,0,60000,2,2,122,3,2,...,2,2,1,2,3,1,1,13177.008,7601.336,2.96
1,4,9,3,0,50000,0,2,122,2,1,...,2,2,2,2,1,1,1,6140.552,3344.434,2.97
2,4,12,3,0,65000,1,2,122,2,1,...,2,2,1,1,3,1,1,9191.061,6949.498,4.28
3,4,14,5,0,120000,0,2,122,1,8,...,2,2,2,1,2,1,1,7900.035,6446.327,7.13
4,1,4,1,0,30000,0,2,115,2,1,...,1,1,1,1,4,1,1,10875.772,8646.586,1.13


In [4]:
depev_final['DEPEV_A']

0        2
1        2
2        2
3        2
4        1
        ..
31954    2
31955    2
31956    2
31957    1
31958    2
Name: DEPEV_A, Length: 31959, dtype: int64

In [5]:
depev_data = depev_final.sample(frac=1)

train = depev_data.sample(frac=0.8)
print('Train:' + str(train.shape))

validate = depev_data.sample(frac=0.1)
print('Validate:' + str(validate.shape))

test = depev_data.sample(frac=0.1)
print('Test:' + str(test.shape))

label_column = 'DEPEV_A'
depev_label = depev_data[['DEPEV_A']].to_numpy()

Train:(25567, 163)
Validate:(3196, 163)
Test:(3196, 163)


In [6]:
# Split into train val test
x_train = train.drop(['DEPEV_A'], axis=1)
x_train = np.asarray(x_train).astype(np.float32)
y_train = train[['DEPEV_A']].to_numpy()
y_train = np.asarray(y_train).astype(np.float32)

x_val = validate.drop(['DEPEV_A'], axis=1)
x_val = np.asarray(x_val).astype(np.float32)
y_val = validate[['DEPEV_A']].to_numpy()
y_val = np.asarray(y_val).astype(np.float32)

x_test = test.drop(['DEPEV_A'], axis=1)
x_test = np.asarray(x_test).astype(np.float32)
y_test = test[['DEPEV_A']].to_numpy()
y_test = np.asarray(y_test).astype(np.float32)

print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
print(x_test.shape)
print(y_test.shape)

(25567, 162)
(25567, 1)
(3196, 162)
(3196, 1)
(3196, 162)
(3196, 1)


In [9]:
xsize=162
ysize=1

t_model = Sequential()
t_model.add(Dense(100, activation="relu", input_shape=(xsize,)))
t_model.add(Dense(50, activation="relu"))
t_model.add(Dense(ysize))
t_model.compile(
    loss="mean_squared_error",
    optimizer=Adam(lr=0.001),
    metrics=[metrics.mae])

In [10]:
epochs = 500
batch = 128

# cols = list(train.columns)
# cols.remove(label_column)
history = t_model.fit(
    x_train, y_train,
    batch_size=batch,
    epochs=epochs,
    shuffle=True,
    verbose=1,
    validation_data=(x_val,y_val)
)
score = t_model.evaluate(x_test, y_test, verbose=0)

Epoch 1/500
200/200 [==============================] - 1s 2ms/step - loss: 477538.0625 - mean_absolute_error: 211.6191 - val_loss: 556.7498 - val_mean_absolute_error: 18.7501
Epoch 2/500
200/200 [==============================] - 0s 2ms/step - loss: 170.4899 - mean_absolute_error: 8.9001 - val_loss: 92.4801 - val_mean_absolute_error: 6.4466
Epoch 3/500
200/200 [==============================] - 0s 2ms/step - loss: 90.5157 - mean_absolute_error: 6.2817 - val_loss: 56.3489 - val_mean_absolute_error: 4.9214
Epoch 4/500
200/200 [==============================] - 0s 2ms/step - loss: 61.5254 - mean_absolute_error: 5.3016 - val_loss: 48.5890 - val_mean_absolute_error: 4.6729
Epoch 5/500
200/200 [==============================] - 0s 2ms/step - loss: 45.7450 - mean_absolute_error: 4.4799 - val_loss: 32.4050 - val_mean_absolute_error: 3.5032
Epoch 6/500
200/200 [==============================] - 0s 2ms/step - loss: 55.1210 - mean_absolute_error: 4.5005 - val_loss: 54.3693 - val_mean_absolute_err

In [14]:
print("R2 score =", round(model.r2_score(x_test, y_test), 2))

NameError: name 'model' is not defined

In [ ]:
from matplotlib.pyplot import figure
figure(figsize=(8, 6))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.ylim([0, 10000])
plt.show()